In [ ]:
import numpy as np
import os
from imutils import paths
import csv

from utils_data import federate_data_iid, shuffle, federate_data_iid_balanced

In [ ]:
path = "../data/Revisadas-Clasificadas-Recortadas"

train_prop = 0.8
num_nodes = 9

for s in [1, 2, 3]:
    np.random.seed(s)

    csv_file_path = "../partitions/partition_iid_"+str(num_nodes)+"nodes_"+str(s)+".csv"

    image_paths = list(paths.list_images(path))

    train_dim = int(train_prop * len(image_paths))

    new_image_paths = shuffle(image_paths)

    train_image_paths = new_image_paths[0:train_dim]
    test_image_paths = new_image_paths[train_dim:]

    federated_data = federate_data_iid(train_image_paths, num_nodes)

    with open(csv_file_path, mode='w') as csv_file:
        fieldnames = ['name', 'class', 'set', 'node']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        set_w = 'test'
        node = '-1'
        for image_path in test_image_paths:
            name = image_path.split(os.path.sep)[-1].split('.')[0]
            label = image_path.split(os.path.sep)[-2]
            writer.writerow({'name': name, 'class': label, 'set': set_w, 'node': node})

        
        set_w = 'train'
        for n in range(num_nodes):
            node = str(n)
            for image_path in federated_data[n]:
                name = image_path.split(os.path.sep)[-1].split('.')[0]
                label = image_path.split(os.path.sep)[-2]
                writer.writerow({'name': name, 'class': label, 'set': set_w, 'node': node})

In [ ]:
csv_file = "../partitions/partition_iid_3nodes_1.csv"
with open(csv_file_path, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count = line_count + 1
        else:
            image_path = path + os.sep + row['class'] + os.sep + row['name'] + '.jpg'